In [1]:
import tensorflow as tf

from tensorflow.python.ops.ragged import ragged_tensor
from tensorflow.python.ops.ragged import ragged_functional_ops
from tensorflow.python.ops import gen_string_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import string_ops
from tensorflow.python.ops.ragged import ragged_string_ops
from tensorflow.python.framework import ops
from tensorflow.python.util import compat
from tensorflow.python.data.ops import dataset_ops


MAX_SEQUENCE_LENGTH = 50
DEFAULT_STRIP_REGEX = r'[’!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']'

def _preprocess(inputs):
    if ragged_tensor.is_ragged(inputs):
        lowercase_inputs = ragged_functional_ops.map_flat_values(
            gen_string_ops.string_lower, inputs)
        lowercase_inputs = array_ops.identity(lowercase_inputs)
    else:
        lowercase_inputs = gen_string_ops.string_lower(inputs)
    inputs = string_ops.regex_replace(lowercase_inputs, DEFAULT_STRIP_REGEX,
                                        " ")
    tokens = ragged_string_ops.string_split_v2(inputs)
    return tokens

def Preprocess_text(sen):
    token = _preprocess(sen)
    words = token.to_tensor(shape=(None, MAX_SEQUENCE_LENGTH))
    init = tf.lookup.TextFileInitializer(filename=VOCAB_FILE_PATH, 
                              key_dtype=tf.string, 
                              key_index=0,
                              value_dtype=tf.int64, 
                              value_index=1,
                              vocab_size=None,
                              delimiter=',')
    table = tf.lookup.StaticVocabularyTable(initializer=init, num_oov_buckets=1)
    word2numbers = table.lookup(words)
    return word2numbers

In [2]:
VOCAB_FILE_PATH = 'saved_model/assets/vocab.txt'
saved_model_path = 'saved_model'

In [3]:
loaded_model = tf.saved_model.load(saved_model_path)
infer = loaded_model.signatures["serving_default"]

In [4]:
sen = [
           "Taste is too good. Loved the way they treated us. Ambience is good.",
           "took my money and not delivered my food, without delivering they closed \
            it as delivered till now I didn't receive my money.",
           "not bad. guava and ginger ice cream was different but a bit syrupy with some candied strawberries. \
            i did not like the thick syrup . But overall nothing bad."
]

In [5]:
sen_batch = Preprocess_text(sen)

In [6]:
result = infer(tf.constant(sen_batch))
list(tf.argmax(result['dense_1'], axis=-1))

[<tf.Tensor: shape=(), dtype=int64, numpy=4>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>]